In [1]:
# %pip install -q sentencepiece
# %pip install -q protobuf
# %pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git
# %pip install -q scipy
# %pip install -q -U trl

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel, PeftConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset, concatenate_datasets
import pandas as pd
import math

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
from torch import nn
import re
import ast
import numpy as np

class CustomSFTTrainer(SFTTrainer):
    def __init__(self, tokenizer, *args, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.tokenizer = tokenizer

    def compute_loss(self, model, inputs, return_outputs=False):
        inputs_copy = inputs.copy()
        labels = inputs_copy.pop("labels")
        outputs = model(**inputs_copy)
        logits = outputs.logits
        
        # Compute the loss using the superclass's compute_loss method
        super_loss, _ = super().compute_loss(model, inputs, return_outputs=True)
        print("\npreloss:", super_loss.item())
        loss = super_loss
        
        wrong_proportion = 1
        
        labels_decoded = decoded = self.tokenizer.batch_decode(labels.tolist())
        labels_result = labels_decoded[0]
        
        try:
            target_match = re.search(r"Test_1_Output=\[\[.*?\]\]", labels_result)
            target_array_string = target_match.group(0).replace("Test_1_Output=", "")
            target_array = np.array(ast.literal_eval(target_array_string))
            
            # Apply softmax to logits to get probabilities
            probs = torch.softmax(logits, dim=-1)
            # Get the predicted tokens (highest probability)
            _, predictions = torch.max(probs, dim=-1)
            
            decoded = self.tokenizer.batch_decode(predictions.tolist())
            predictions_result = decoded[0]
            match1 = re.search(r"Test_1_Output=\[\[.*?\]\]", predictions_result)
            predicted_array_string = ""
            if match1:
                predicted_array_string = match1.group(0).replace("Test_1_Output=", "")
                print("matched Test_1_Output=[[*?]]")
            else:
                loss *= 2
                print("Did not match Test_1_Output=[[*?]]..predicted")
                print(predictions_result)

            if predicted_array_string != "":
                try:
                    predicted_array = np.array(ast.literal_eval(predicted_array_string))
                    if predicted_array.shape == target_array.shape:
                        print("correct shape")
                        loss *= loss
                        equal_elements_count = np.sum(np.equal(target_array, predicted_array))
                        total_elements_count = target_array.size + 1
                        wrong_elements_count = total_elements_count - equal_elements_count
                        wrong_proportion = wrong_elements_count / total_elements_count
                        loss *= wrong_proportion * wrong_proportion
                        are_equal = np.array_equal(predicted_array, target_array)
                        if are_equal:
                            print("are equal!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                            print("target")
                            print(target_array)
                            print("predicted")
                            print(predicted_array)
                            loss *= loss
                    else:
                        print("incorrect shape")
                        print("target")
                        print(target_array)
                        print("predicted")
                        print(predicted_array)
                except:
                    print("failed to make predicted array")
                    print(predicted_array_string)
        except:
            print("failed to make target array")
            print(labels_result)
        
        print("wp:", wrong_proportion, "  loss:", loss.item())

        return (loss, outputs) if return_outputs else loss

In [4]:
def generate_prompt(data_point):
    user_message = data_point["instruction"]
    assistant_message = data_point["output"]
    text = f"<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n{assistant_message}<|im_end|>"
    return text

In [5]:
base_model_name = "merged_models/Orca_2_finetune_04_merge"
new_adapter_name = "Orca-2-13b-4bit-finetune-on-ARC-15"

# Load the entire model on the GPU 0
device_map = {"": 0}

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [6]:
# Load the data using pandas
data_file = "data/core_ARC_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset3 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/ARC_augmented_training_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset1 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/ARC_augmented_evaluation_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset2 = Dataset.from_pandas(df)

# Generate the "prompt" column for each dataset
dataset1 = dataset1.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset2 = dataset2.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset3 = dataset3.map(lambda data_point: {"prompt": generate_prompt(data_point)})

# # Assuming dataset1 and dataset2 are instances of Dataset from the `datasets` library
dataset = concatenate_datasets([dataset1, dataset2, dataset3])

Map:   0%|          | 0/11240 [00:00<?, ? examples/s]

Map:   0%|          | 0/72504 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
def main():
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map=device_map,
    )
    model.config.use_cache = False
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)  

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, add_eos_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

    per_device_train_batch_size = 1
    gradient_accumulation_steps = 1
    max_seq_length = 5000

    output_dir = "results/" + new_adapter_name

    steps_per_epoch = len(dataset) // (
        per_device_train_batch_size * gradient_accumulation_steps
    )
    print("Steps:", steps_per_epoch)

    # Set training parameters
    training_arguments = TrainingArguments(
        output_dir=output_dir,
        max_steps=steps_per_epoch,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        optim="paged_adamw_32bit",
        lr_scheduler_type="cosine",
#         weight_decay=0.01,
        save_strategy="steps",
        evaluation_strategy="no",
        save_steps=1000,
        logging_steps=1,
        learning_rate=5e-6,
        fp16=True,
        warmup_steps=0.03,
        group_by_length=True,
        gradient_checkpointing=True,
    )

    trainer = CustomSFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        dataset_text_field="prompt",
        max_seq_length=max_seq_length,
        tokenizer=tokenizer,
        args=training_arguments,
    )

    # Train model
    trainer.train()

    # Save trained model
    adapter = "adapters/" + new_adapter_name
    trainer.model.save_pretrained(adapter)

In [8]:
main()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Steps: 93744


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/93744 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



preloss: 0.6165382266044617
Did not match Test_1_Output=[[*?]]..predicted
<|im_start|>user
Train_1_Input=[[0,5,2,6,6,6,6,7,9,7,7,7,5,2,6,1,1,1,1,1,1,1,1,1],[5,1,1,1,1,1,1,1,1,1,1,1],[1,1,1,1,1,1,1,1,1,9,1,1],[1,1,1,1,1,1,1,1,8,1,1,1],[1,1,1,1,1,1,1,1,1,1,0,1],[1,1,1,1,1,1,9,1,9,9,5,9],[1,1,1,1,1,1,1,1,1,1,9,1],[1,1,1,1,1,1,1,1,1,1,8,1],[1,1,1,9,9,9,1,9,9,9,0,0],[9,1,9,1,1,1,1,9,9,1,1,1],[1,1,1,1,1,1,1,1,1,1,1,1],[1,1,1,1,1,9,1,1,1,1,1,0],[1,1,1,1,1,1,1,1,1,1,3,3],[1,1,1,1,1,1,1,1,0,0,0,0],[1,1,0,9,1,1,1,0,3,9,9,9],[1,9,0,9,9,1,1,1,1,1,1,1],[1,1,9,1,1,1,1,1,1,1,1,1],[1,1,1,1,1,1,1,1,1,1,1,9],[1,1,1,1,1,1,1,1,1,1,1,1],[1,1,9,1,1,1,1,1,1,1,1,1],[ Train_1_Output=[[1,0,0,0,0,0,1,0,0,2,0,0],[0,0,0,0,0,0,0,4,9,0,0,0],[0,0,0,0,0,0,1,6,0,0,0,0],[0,0,0,0,0,8,9,8,2,8,0,0],[0,5,1,4,9,8,0,4,0,0,0,0],[0,0,1,2,0,0,0,0,0,0,0,0],[0,9,9,8,2,0,0,0,0,0,0,0],[0,5,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,

Step,Training Loss



preloss: 0.38128259778022766
matched Test_1_Output=[[*?]]
correct shape
wp: 0.030470914127423823   loss: 0.00013497860345523804


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor []], which is output 0 of MulBackward0, is at version 2; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).